# Получение информации из базы данных с помощью SQL

**Краткая предыстория**  

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель исследования**  

Проанализировать базу данных сервиса для чтения книг по подписке.

**Задания**  
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Оглавление

[1. Подключение к базе данных, ознакомление с таблицами](#section1)  
[2. Решение задач](#section2)  
    [2.1 Количество книг вышедших после 1 января 2000 года](#section2_1)  
    [2.2 Количество обзоров и средняя оценка книг](#section2_2)  
    [2.3 Издательство, которое выпустило наибольшее число книг толще 50 страниц](#section2_3)  
    [2.4 Автор с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)](#section2_4)     
    [2.5 Среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#section2_5)  

## 1. Подключение к базе данных, ознакомление с таблицами
<a id='section1'> <a/>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# Таблица books. Содержит данные о книгах
query = ''' SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)

display(books.head(3))
books.info()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [3]:
# Таблица authors. Содержит данные об авторах
query = ''' SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)

display(authors.head(3))
authors.info()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [4]:
# Таблица publishers. Содержит данные об издательствах
query = ''' SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)

display(publishers.head(3))
publishers.info()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [5]:
# Таблица ratings. Содержит данные о пользовательских оценках книг
query = ''' SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)

display(ratings.head(3))
ratings.info()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [6]:
# Таблица reviews. Содержит данные о пользовательских обзорах на книги
query = ''' SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)

display(reviews.head(3))
reviews.info()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


## 2. Решение задач
<a id='section2'> <a/>

**1. Количество книг вышедших после 1 января 2000 года**
<a id='section2_1'> <a/>

In [7]:
query = ''' SELECT COUNT(title) AS count_book
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
books_after_2000 = pd.io.sql.read_sql(query, con = engine)

books_after_2000.head()

,count_book
0,819


В базе данных находится информация о 819 книгах, вышедших после 1 января 2000 года (включая 1 января - 821 книга)

**2. Количество обзоров и средняя оценка книг**
<a id='section2_2'> <a/>

In [8]:
query = ''' SELECT
                books.title AS title,
                rating.avg_rating AS average_rating,
                review.cnt_review AS number_of_review
            FROM books
            INNER JOIN (SELECT
                            book_id,
                            AVG(rating) AS avg_rating
                        FROM ratings
                        GROUP BY book_id) AS rating ON rating.book_id = books.book_id
            INNER JOIN (SELECT
                            book_id,
                            COUNT(DISTINCT review_id) AS cnt_review
                        FROM reviews
                        GROUP BY book_id) AS review ON review.book_id = books.book_id
        '''
books_rating_and_review = pd.io.sql.read_sql(query, con = engine)

books_rating_and_review.head()

,title,average_rating,number_of_review
0,'Salem's Lot,3.666667,2
1,1 000 Places to See Before You Die,2.500000,1
2,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,1491: New Revelations of the Americas Before C...,4.500000,2
4,1776,4.000000,4


В данной таблице собрана информация о средней оценке каждой книги и количестве ревью на нее.

**3. Издательство, которое выпустило наибольшее число книг толще 50 страниц**
<a id='section2_3'> <a/>

In [9]:
query = ''' SELECT
                publishers.publisher AS publisher,
                COUNT(books.title) AS number_of_books
            FROM books
            INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
            WHERE books.num_pages > 50
            GROUP BY publishers.publisher
            ORDER BY number_of_books DESC
            LIMIT 1
        '''
publisher_number_of_books = pd.io.sql.read_sql(query, con = engine)

publisher_number_of_books.head()

,publisher,number_of_books
0,Penguin Books,42


Наибольшее число книг, из находящихся в базе, выпустило издательство Penguin Books. При подсчете учитывались книги больше 50 страниц.

**4. Автор с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)**
<a id='section2_4'> <a/>

In [10]:
query = ''' SELECT
                authors.author AS author,
                AVG(SUBQ.avg_rating) AS average_rating
            FROM books
            INNER JOIN authors ON authors.author_id = books.author_id
            INNER JOIN (SELECT
                            book_id,
                            AVG(rating) AS avg_rating
                        FROM ratings
                        GROUP BY book_id
                        HAVING COUNT(rating) > 50) AS SUBQ ON SUBQ.book_id = books.book_id
            GROUP BY authors.author
            ORDER BY average_rating DESC
            LIMIT 1
        '''
authors_rating = pd.io.sql.read_sql(query, con = engine)

authors_rating.head()

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Наиболее высокоонцененным автором в сервисе является Джоан Роулинг с рейтингом в 4.28 балла.

**5.  Среднее количество обзоров от пользователей, которые поставили больше 50 оценок**
<a id='section2_5'> <a/>

In [11]:
query = ''' SELECT
                AVG(count_review.cnt_review) AS average_count_review
            FROM (SELECT
                      username,
                      COUNT(review_id) AS cnt_review
                  FROM reviews
                  GROUP BY username) AS count_review
            INNER JOIN (SELECT
                            username,
                            COUNT(rating_id) AS cnt_rating
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating) > 50) AS users ON users.username = count_review.username
        '''
average_count_review = pd.io.sql.read_sql(query, con = engine)

display(average_count_review)

,average_count_review
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.3